<a href="https://colab.research.google.com/github/charliecarver/cosc247/blob/master/deliverable-3/deliverable-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Data

In [2]:
# Load CSV files from remote repo
import requests
import zipfile
import io

r = requests.get('https://github.com/charliecarver/cosc247/blob/master/datasets.zip?raw=true')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
testPath = 'Test.csv'
trainPath = 'Train.csv'

## Single-File Deliverable

In [6]:
# Import required libraries
import pandas as pd
import numpy as np
import scipy.sparse
import string
from ast import literal_eval
import nltk.tokenize
import nltk.stem.porter
import math
from nltk.corpus import stopwords
import sklearn.metrics
import statistics
import sklearn.naive_bayes
import sklearn.feature_extraction.text
from sklearn import model_selection
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import sklearn.model_selection
import sklearn.metrics
import sklearn.naive_bayes
import sklearn.tree
import sklearn.linear_model
from sklearn.metrics import classification_report, confusion_matrix
from timeit import default_timer as timer
from matplotlib import pyplot as plt
from matplotlib import ticker
from sklearn.tree import export_graphviz
import graphviz
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn.linear_model
import sklearn.model_selection
import statistics
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Flags
useTestCSV = False
NGRAM_SIZE = 2
COMMON_WORD_THRESHOLD = 10
stemmer = nltk.stem.porter.PorterStemmer()

def preprocessForTextClassification(df):
    df['reviewText'] = df['reviewText'].fillna("")
    df['summary'] = df['summary'].fillna("")

    P = df.groupby('amazon-id').agg({
        'reviewText': ' '.join,
        'summary': ' '.join,
    })

    P['reviewText'] = P['reviewText'] + " " + P['summary']

    return P


# Train text classifier
def trainTextFrequency(df):
    P = preprocessForTextClassification(df)

    vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,NGRAM_SIZE))
    X1 = vectorizer.fit_transform(P['reviewText'])

    return X1, vectorizer

def getTextMatrix(df, word_indices):
    P = preprocessForTextClassification(df)

    X1 = word_indices.transform(P['reviewText'])
    return X1

# function for normalization
def normalize_column_data(input_data):
    for feature in input_data:    
        input_data[feature] = (input_data[feature]-input_data[feature].min())/(input_data[feature].max()-input_data[feature].min())

# Process numerical data
def processNumerical(df):

    # Drop text data
    df = df.drop(columns=['title', 'categories', 'songs', 'related', 'reviewTime'])

    # Drop columns that need more time to process
    df = df.drop(columns=['label', 'first-release-year'])

    # Transform helpful into "ratio" of being helpful
    df['helpful'] = df['helpful'].apply(lambda x: np.nan if literal_eval(x)[1]== 0 else literal_eval(x)[0]/literal_eval(x)[1])
    df['helpful'].fillna((df['helpful'].median()), inplace=True)

    # Convert categorical data to their own features
    # df = df.join(pd.get_dummies(df['root-genre']))
    df = df.drop(columns=['root-genre'])

    # Return processed data
    return df

# Load data
try:
    testPath
except NameError:
    # Default paths of CSV files
    print('Loading files from default locations')
    testPath = 'Test.csv'
    trainPath = 'Train.csv'

# Load dataframes
dfTrain = pd.read_csv(trainPath)
if useTestCSV: dfTest = pd.read_csv(testPath)

# Train text classifier on training data
trainingTextMatrix, wordIndices = trainTextFrequency(dfTrain)

# Process textual data
if useTestCSV:
    testTextMatrix = getTextMatrix(dfTest, wordIndices)

# Process numerical data
dfTrain = processNumerical(dfTrain)
if useTestCSV: dfTest = processNumerical(dfTest)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/charlescarver/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/charlescarver/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/charlescarver/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/charlescarver/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
# Aggregate training
isAwesome = lambda x: 1 if np.mean(x) > 4.5 else 0
trainData = dfTrain.groupby('amazon-id').agg({
    'unixReviewTime': 'mean',
    'price': 'mean',
    'overall': isAwesome,
    'salesRank': 'mean',
    'helpful': 'mean',
})

# normalization for numerical features
normalize_column_data(trainData)

# Aggregate testing data and split into dependent/independent vars
if useTestCSV:
    testData = dfTest.groupby('amazon-id').agg({
        'unixReviewTime': 'mean',
        'price': 'mean',
        'salesRank': 'mean',
        'helpful': 'mean',
    })
    normalize_column_data(testData)

    ytrain = trainData['overall'].to_numpy()
    Xtrain = scipy.sparse.hstack(
        (trainingTextMatrix, scipy.sparse.csr_matrix(trainData.drop(columns='overall').to_numpy()))
    )
    Xtrain = scipy.sparse.csr_matrix(Xtrain)
    testIndex = testData.index
    Xtest = scipy.sparse.hstack(
        (testTextMatrix, scipy.sparse.csr_matrix(testData.to_numpy()))
    )
    Xtest = scipy.sparse.csr_matrix(Xtest)
else:
    #Xtrain = scipy.sparse.csr_matrix(scipy.sparse.hstack(
    #    (trainingTextMatrix, scipy.sparse.csr_matrix(trainData.drop(columns='overall').to_numpy()))
    #))
    Xtrain = scipy.sparse.csr_matrix(scipy.sparse.hstack(
        (trainingTextMatrix, scipy.sparse.csr_matrix(trainData['helpful'].to_numpy().reshape(-1,1)))
    ))

    ytrain = trainData['overall'].to_numpy()
    #Xtrain, Xtest, ytrain, ytest = sklearn.model_selection.train_test_split(Xtrain, trainData['overall'].to_numpy(), test_size=0.3, shuffle=True)

In [8]:
# Run ML
kf = sklearn.model_selection.KFold(n_splits=10, random_state=42, shuffle=True)
f1_vals = []

for train_index, test_index in kf.split(Xtrain):
    x_train, x_test = Xtrain[train_index], Xtrain[test_index]
    y_train, y_test = ytrain[train_index], ytrain[test_index]

    #clf = sklearn.naive_bayes.MultinomialNB()
    #clf = sklearn.tree.DecisionTreeClassifier(max_depth=10)
    clf = sklearn.linear_model.LogisticRegression(max_iter=100000, class_weight='balanced')
    clt = clf.fit(x_train, y_train)

    f1 = sklearn.metrics.f1_score(y_test, clt.predict(x_test), average='weighted')
    print("F1 {}".format(f1))
    f1_vals.append(f1)
 
print("Mean F1: ", statistics.mean(f1_vals))
gnb = sklearn.linear_model.LogisticRegression(max_iter=100000, class_weight='balanced')
gnbTrained = gnb.fit(Xtrain, ytrain)
ypreds = gnbTrained.predict(Xtest)

# only for numerical data best model

# Testing
if not useTestCSV:
    print(sklearn.metrics.f1_score(ytest, ypreds, average='weighted'))

# Output CSV file with predictions
if useTestCSV:

    # Output predictions for deliverable
    output = pd.DataFrame({'amazon-id': testIndex, 'Awesome': ypreds})
    output.to_csv('./Product_Predictions.csv')

F1 0.7372399933059709
F1 0.7196273026611997
F1 0.7402843601895734
F1 0.7211360935906432
F1 0.7290402235615303
F1 0.7084146943000765
F1 0.7248576850094877
F1 0.7522457382915118
F1 0.7317060620394247
F1 0.731465443313517
Mean F1:  0.7296017596262935


ValueError: X has 2576580 features per sample; expecting 2576577